In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

pasta_dados = '../dados/'

df_empreend = pd.read_csv(pasta_dados + 'tratados/simu_carteira_empreendimentos.csv')
df_acidentes = pd.read_csv(pasta_dados + 'tratados/simu_acidentes_de_transportes.csv')

In [2]:
df_empreend.shape

(71248, 121)

In [3]:
df_empreend.columns

Index(['cod_mdr', 'cod_operacao', 'cod_saci', 'cod_convenio_siafi',
       'cod_ag_operador', 'origem', 'dsc_concedente', 'dsc_fonte',
       'dsc_unidade', 'cod_proposta',
       ...
       'aux_emp_viaria', 'aux_emp_aero', 'aux_emp_revitalizacao',
       'aux_emp_infraestrutura', 'aux_emp_pac2etapa',
       'aux_emp_faixa_exclusiva', 'aux_emp_rapido', 'aux_emp_sarjeta',
       'aux_emp_nan', 'new_empreend'],
      dtype='object', length=121)

## Agrupamento dos empreendimentos por município e ano

In [4]:
df_empreend[['vlr_investimento', 'pop_beneficiada']].isna().sum()

vlr_investimento    1036
pop_beneficiada        0
dtype: int64

In [5]:
df_empreend[df_empreend['vlr_investimento'].isnull() &df_empreend['Código IBGE'].notnull() ]

,cod_mdr,cod_operacao,cod_saci,cod_convenio_siafi,cod_ag_operador,origem,dsc_concedente,dsc_fonte,dsc_unidade,cod_proposta,dsc_situacao_contrato_mdr,bln_carteira_mdr,bln_carteira_mdr_ativo,dsc_situacao_objeto_mdr,dsc_paralisada_mdr,bln_ativo,bln_ativo_boolean,cod_id_fonte,cod_id_pac,programa,acao,uf_SIGLA_UF,mun_MUNNOMEX,Código IBGE,empreendimento,vlr_repasse_financiamento,vlr_contrapartida,vlr_empenhado,vlr_desembolsado,vlr_desbloqueado,vlr_desembolsado_19,vlr_desembolsado_20,vlr_desembolsado_21,situacao_obra,situacao_obra_base_validacao,situacao_contrato,prc_fisico,dte_assinatura,dte_inicio_obra,dte_fim_obra,dte_ult_desembolso,dte_ult_desbloqueio,dte_ult_ref_bm,vlr_ult_desembolso,vlr_ult_repasse_desbloqueado,vlr_ult_contrapartida_desbloqueado,agente_financeiro,agente_financeiro_saci,dias_paralisados,situacao_obra_base_validacao_secex,situacao_obra_mdr,pop_beneficiada,emp_gerado,vlr_investimento,vlr_ult_investimento_desbloqueado,vlr_repasse_financiamento_num,vlr_contrapartida_num,vlr_investimento_num,vlr_empenhado_num,vlr_desembolsado_num,vlr_desbloqueado_num,vlr_desembolsado_19_num,vlr_desembolsado_20_num,vlr_desembolsado_21_num,vlr_ult_desembolso_num,vlr_ult_repasse_desbloqueado_num,vlr_ult_contrapartida_desbloqueado_num,vlr_ult_investimento_desbloqueado_num,ano_assinatura,ano_inicio_obra,ano_fim_obra,ano_ult_desembolso,ano_ult_desbloqueio,ano_ult_ref_bm,ano,Populacao,Região,uf_REGIAO,Município,mun_coordenadas,mun_MUNNOME,mun_AMAZONIA,mun_FRONTEIRA,mun_CAPITAL,uf_CODIGO_UF,mun_LATITUDE,mun_LONGITUDE,mun_ALTITUDE,mun_AREA,mun_codigo_adotado,uf_NOME_UF,aux_emp_implantacao,aux_emp_pavimentacao,aux_emp_refrota,aux_emp_sinalizacao,aux_emp_calcamento,aux_emp_ciclovia,aux_emp_corredor,aux_emp_drenagem,aux_emp_ferrovia,aux_emp_transporte,aux_emp_brt,aux_emp_vlt,aux_emp_qualificacao,aux_emp_mobilidade_urbana,aux_emp_aquisicao,aux_emp_polongamento,aux_emp_ampliacao,aux_emp_recapeamento,aux_emp_linha,aux_emp_metro,aux_emp_viaria,aux_emp_aero,aux_emp_revitalizacao,aux_emp_infraestrutura,aux_emp_pac2etapa,aux_emp_faixa_exclusiva,aux_emp_rapido,aux_emp_sarjeta,aux_emp_nan,new_empreend


In [6]:
import numpy as np

aux_emp_columns = df_empreend.filter(like='aux_emp').columns

agg_dict = {
    'vlr_investimento': 'sum',
    'Populacao': 'mean',
    'pop_beneficiada': 'sum',
    'Código IBGE': 'size'
}

# Para calcular o total de investimentos em cada categoria de empreendimentos, vamos primeiro criar uma coluna para cada categoria
# que é igual ao investimento daquele empreendimento se coluna não nula e igual a zero se coluna não zero
# Em seguida, quando agruparmos as colunas vamos somar o valor de investimento em cada categoria
for column in aux_emp_columns:
    agg_dict[column] = lambda x: x.notnull().sum()

    new_column_name = column + '_vlr_investimento'
    
    # Calcula a soma dos valores da coluna "vlr_investimento" onde a coluna aux_emp correspondente não é nula
    df_empreend[new_column_name] = df_empreend.apply(lambda row: row['vlr_investimento'] if pd.notnull(row[column]) else 0, axis=1)
    
    agg_dict[new_column_name] = 'sum'

# Cria colunas dummy para cada valor distinto na coluna 'programa' - Faremos o mesmo que fizemos com as categorias de empreendimentos
programa_dummies = pd.get_dummies(df_empreend['programa'], prefix='programa')

df_empreend = pd.concat([df_empreend, programa_dummies], axis=1)

for column in programa_dummies.columns:
    agg_dict[column] = 'sum'
    
    new_column_name = column + '_vlr_investimento'
    
    # Calcula a soma dos valores da coluna "vlr_investimento" onde a coluna 'programa' corresponde a cada valor distinto
    df_empreend[new_column_name] = df_empreend.apply(lambda row: row['vlr_investimento'] if row['programa'] == column.replace('programa_', '') else 0, axis=1)
    
    agg_dict[new_column_name] = 'sum'

df_agrupado_municipio_empreend = df_empreend.groupby(['Código IBGE', 'ano']).agg(agg_dict)

df_agrupado_municipio_empreend.rename(columns={'Código IBGE': 'num_total_empreendimentos' }, inplace=True)

# Também vamos calcular o valor de investimento e a população beneficiada per capita
df_agrupado_municipio_empreend['vlr_investimento_per_capita'] = df_agrupado_municipio_empreend['vlr_investimento'] / df_agrupado_municipio_empreend['Populacao']
df_agrupado_municipio_empreend['pop_beneficiada_per_capita'] = df_agrupado_municipio_empreend['pop_beneficiada'] / df_agrupado_municipio_empreend['Populacao']

df_agrupado_municipio_empreend

vlr_investimento  Populacao  pop_beneficiada  \
Código IBGE ano                                                    
1100015.0   2011.0      1.530000e+06    24228.0           1841.0   
            2021.0      1.000000e+06    22516.0           1193.0   
1100023.0   1997.0      9.983458e+05    70956.0           1000.0   
            2001.0      6.598815e+05    76371.0            746.0   
            2005.0      1.379548e+06    85031.0           1187.0   
...                              ...        ...              ...   
5300108.0   2018.0      4.426031e+07  2974703.0          50705.0   
            2019.0      1.020757e+06  3015268.0           1268.0   
            2020.0      1.199639e+08  3055149.0          84612.0   
            2021.0      9.710518e+06  3094325.0          12056.0   
            2022.0      6.322037e+06  2923369.0           7780.0   

                    num_total_empreendimentos  aux_emp_implantacao  \
Código IBGE ano                                                      
1100015.0   2011.0                          2                    0   
            2021.0                          1                    0   
1100023.0   1997.0                          1                    0   
            2001.0                          1                    0   
            2005.0                          4                    2   
...                                       ...                  ...   
5300108.0   2018.0                          8                    2   
            2019.0                          1                    0   
            2020.0                          7                    3   
            2021.0                          3                    0   
            2022.0                          2                    0   

                    aux_emp_implantacao_vlr_investimento  \
Código IBGE ano                                            
1100015.0   2011.0                                  0.00   
            2021.0                                  0.00   
1100023.0   1997.0                                  0.00   
            2001.0                                  0.00   
            2005.0                             694240.60   
...                                                  ...   
5300108.0   2018.0                           38486221.13   
            2019.0                                  0.00   
            2020.0                           57178875.42   
            2021.0                                  0.00   
            2022.0                                  0.00   

                    aux_emp_pavimentacao  \
Código IBGE ano                            
1100015.0   2011.0                     2   
            2021.0                     0   
1100023.0   1997.0                     1   
            2001.0                     1   
            2005.0                     3   
...                                  ...   
5300108.0   2018.0                     3   
            2019.0                     1   
            2020.0                     3   
            2021.0                     3   
            2022.0                     2   

                    aux_emp_pavimentacao_vlr_investimento  aux_emp_refrota  \
Código IBGE ano                                                              
1100015.0   2011.0                             1530000.00                0   
            2021.0                                   0.00                0   
1100023.0   1997.0                              998345.83                0   
            2001.0                              659881.54                0   
            2005.0                             1063361.26                0   
...                                                   ...              ...   
5300108.0   2018.0                             3725213.98                0   
            2019.0                             1020756.76                0   
            2020.0                            43397717.53                0   
            

In [7]:
df_agrupado_municipio_empreend.shape

(45979, 76)

## Combinação da base de empreendimentos com a de acidentes

In [8]:
df_acidentes

,Código IBGE,ano,Região,Pedestre_mortes,Ciclista_mortes,Motociclista_mortes,Ocup_triciclo_motor_mortes,Ocup_automovel_mortes,Ocup_caminhonete_mortes,Ocup_veic_transp_pesado_mortes,Ocup_onibus_mortes,Outros_mortes,Populacao_acidentes,total_mortes,taxa_mun_mortes,taxa_mun_pedestre_mortes,taxa_mun_ciclista_mortes,taxa_mun_motociclista_mortes,taxa_mun_automovel_mortes,taxa_mun_veiculo_pesado_mortes,taxa_mun_onibus_mortes,taxa_mun_outros_mortes,Pedestre_feridos,Ciclista_feridos,Motociclista_feridos,Ocup_triciclo_motor_feridos,Ocup_automovel_feridos,Ocup_caminhonete_feridos,Ocup_veic_transp_pesado_feridos,Ocup_onibus_feridos,Outros_feridos,total_feridos,taxa_mun_feridos,taxa_mun_pedestre_feridos,taxa_mun_ciclista_feridos,taxa_mun_motociclista_feridos,taxa_mun_automovel_feridos,taxa_mun_veiculo_pesado_feridos,taxa_mun_onibus_feridos,taxa_mun_outros_feridos,Populacao_populacao,taxa_mun_triciclo_mortes,taxa_mun_caminhonete_mortes,taxa_mun_triciclo_feridos,taxa_mun_caminhonete_feridos
0,1100015,2010,N,2,0,5,0,0,0,0,0,0,24392,7.0,0.028698,0.008199,0.000000,0.020499,0.000000,0.000000,0.000000,0.000000,0,0,13,0,0,0,0,0,0,13.0,0.053296,0.000000,0.000000,0.053296,0.000000,0.000000,0.000000,0.000000,24392.0,0.0,0.000000,0.000000,0.000000
1,1100023,2010,N,0,1,2,0,0,0,0,1,29,90353,33.0,0.036523,0.000000,0.001107,0.002214,0.000000,0.000000,0.001107,0.032096,0,1,37,0,1,1,0,0,3,43.0,0.047591,0.000000,0.001107,0.040950,0.001107,0.000000,0.000000,0.003320,90353.0,0.0,0.000000,0.000000,0.001107
2,1100031,2010,N,0,0,0,0,0,0,0,0,0,6313,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6313.0,0.0,0.000000,0.000000,0.000000
3,1100049,2010,N,2,1,17,0,4,0,0,0,2,78574,26.0,0.033090,0.002545,0.001273,0.021636,0.005091,0.000000,0.000000,0.002545,50,1,5,0,0,0,0,0,1,57.0,0.072543,0.063634,0.001273,0.006363,0.000000,0.000000,0.000000,0.001273,78574.0,0.0,0.000000,0.000000,0.000000
4,1100056,2010,N,3,0,1,0,0,0,0,0,0,17029,4.0,0.023489,0.017617,0.000000,0.005872,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0,0,1.0,0.005872,0.000000,0.005872,0.000000,0.000000,0.000000,0.000000,0.000000,17029.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55695,5222005,2019,CO,1,0,1,0,0,0,0,0,2,13863,4.0,0.028854,0.007213,0.000000,0.007213,0.000000,0.000000,0.000000,0.014427,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13863.0,0.0,0.000000,0.000000,0.000000
55696,5222054,2019,CO,0,0,0,0,2,0,0,0,2,8743,4.0,0.045751,0.000000,0.000000,0.000000,0.022875,0.000000,0.000000,0.022875,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8743.0,0.0,0.000000,0.000000,0.000000
55697,5222203,2019,CO,0,0,1,0,6,0,1,0,0,6171,8.0,0.129639,0.000000,0.000000,0.016205,0.097229,0.016205,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6171.0,0.0,0.000000,0.000000,0.000000
55698,5222302,2019,CO,0,0,0,0,1,0,0,0,2,5821,3.0,0.051538,0.000000,0.000000,0.000000,0.017179,0.000000,0.000000,0.034358,0,0,0,0,0,0,0,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5821.0,0.0,0.000000,0.000000,0.000000


Sabemmos que df_acidentes é único por conjunto Código IBGE e ano

In [9]:
df_agrupado_municipio_empreend.shape

(45979, 76)

In [10]:
df_acidentes.shape

(55700, 45)

In [11]:
list(set(df_agrupado_municipio_empreend.columns).intersection(set(df_acidentes.columns)))

[]

In [12]:
df_merge_empreendimentos_acidentes = pd.merge(df_agrupado_municipio_empreend, df_acidentes, on=['Código IBGE', 'ano'], how='outer')
df_merge_empreendimentos_acidentes.shape

(80979, 121)

Muitas linhas surgiram, pois a interseção de conjuntos Código IBGE e ano que possuem algum empreendimento e ao mesmo tempo dados de mortes e feridos é só de 20700

In [13]:
pd.merge(df_agrupado_municipio_empreend, df_acidentes, on=['Código IBGE', 'ano'], how='inner').shape

(20700, 121)

In [14]:
unique_rows = df_merge_empreendimentos_acidentes.duplicated(subset=['Código IBGE', 'ano'], keep=False)

if unique_rows.sum() == 0:
    print("O conjunto dos valores das colunas 'Código IBGE' e 'ano' é único.")
else:
    print("O conjunto dos valores das colunas 'Código IBGE' e 'ano' não é único.")


O conjunto dos valores das colunas 'Código IBGE' e 'ano' é único.


In [15]:
df_merge_empreendimentos_acidentes

,Código IBGE,ano,vlr_investimento,Populacao,pop_beneficiada,num_total_empreendimentos,aux_emp_implantacao,aux_emp_implantacao_vlr_investimento,aux_emp_pavimentacao,aux_emp_pavimentacao_vlr_investimento,aux_emp_refrota,aux_emp_refrota_vlr_investimento,aux_emp_sinalizacao,aux_emp_sinalizacao_vlr_investimento,aux_emp_calcamento,aux_emp_calcamento_vlr_investimento,aux_emp_ciclovia,aux_emp_ciclovia_vlr_investimento,aux_emp_corredor,aux_emp_corredor_vlr_investimento,aux_emp_drenagem,aux_emp_drenagem_vlr_investimento,aux_emp_ferrovia,aux_emp_ferrovia_vlr_investimento,aux_emp_transporte,aux_emp_transporte_vlr_investimento,aux_emp_brt,aux_emp_brt_vlr_investimento,aux_emp_vlt,aux_emp_vlt_vlr_investimento,aux_emp_qualificacao,aux_emp_qualificacao_vlr_investimento,aux_emp_mobilidade_urbana,aux_emp_mobilidade_urbana_vlr_investimento,aux_emp_aquisicao,aux_emp_aquisicao_vlr_investimento,aux_emp_polongamento,aux_emp_polongamento_vlr_investimento,aux_emp_ampliacao,aux_emp_ampliacao_vlr_investimento,aux_emp_recapeamento,aux_emp_recapeamento_vlr_investimento,aux_emp_linha,aux_emp_linha_vlr_investimento,aux_emp_metro,aux_emp_metro_vlr_investimento,aux_emp_viaria,aux_emp_viaria_vlr_investimento,aux_emp_aero,aux_emp_aero_vlr_investimento,aux_emp_revitalizacao,aux_emp_revitalizacao_vlr_investimento,aux_emp_infraestrutura,aux_emp_infraestrutura_vlr_investimento,aux_emp_pac2etapa,aux_emp_pac2etapa_vlr_investimento,aux_emp_faixa_exclusiva,aux_emp_faixa_exclusiva_vlr_investimento,aux_emp_rapido,aux_emp_rapido_vlr_investimento,aux_emp_sarjeta,aux_emp_sarjeta_vlr_investimento,aux_emp_nan,aux_emp_nan_vlr_investimento,programa_AVANÇAR PÚBLICO,programa_AVANÇAR PÚBLICO_vlr_investimento,programa_EMENDAS,programa_EMENDAS_vlr_investimento,programa_PAC FIN,programa_PAC FIN_vlr_investimento,programa_PAC OGU,programa_PAC OGU_vlr_investimento,programa_PRÓ COMUNIDADE,programa_PRÓ COMUNIDADE_vlr_investimento,programa_SETOR PRIVADO,programa_SETOR PRIVADO_vlr_investimento,vlr_investimento_per_capita,pop_beneficiada_per_capita,Região,Pedestre_mortes,Ciclista_mortes,Motociclista_mortes,Ocup_triciclo_motor_mortes,Ocup_automovel_mortes,Ocup_caminhonete_mortes,Ocup_veic_transp_pesado_mortes,Ocup_onibus_mortes,Outros_mortes,Populacao_acidentes,total_mortes,taxa_mun_mortes,taxa_mun_pedestre_mortes,taxa_mun_ciclista_mortes,taxa_mun_motociclista_mortes,taxa_mun_automovel_mortes,taxa_mun_veiculo_pesado_mortes,taxa_mun_onibus_mortes,taxa_mun_outros_mortes,Pedestre_feridos,Ciclista_feridos,Motociclista_feridos,Ocup_triciclo_motor_feridos,Ocup_automovel_feridos,Ocup_caminhonete_feridos,Ocup_veic_transp_pesado_feridos,Ocup_onibus_feridos,Outros_feridos,total_feridos,taxa_mun_feridos,taxa_mun_pedestre_feridos,taxa_mun_ciclista_feridos,taxa_mun_motociclista_feridos,taxa_mun_automovel_feridos,taxa_mun_veiculo_pesado_feridos,taxa_mun_onibus_feridos,taxa_mun_outros_feridos,Populacao_populacao,taxa_mun_triciclo_mortes,taxa_mun_caminhonete_mortes,taxa_mun_triciclo_feridos,taxa_mun_caminhonete_feridos
0,1100015.0,2011.0,1530000.00,24228.0,1841.0,2.0,0.0,0.0,2.0,1530000.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1530000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1530000.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.150074,0.075986,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24228.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24228.0,0.0,0.000000,0.0,0.0
1,1100015.0,2021.0,1000000.00,22516.0,1193.0,1.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1000000.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.412862,0.052985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [16]:
df_merge_empreendimentos_acidentes.to_csv('../dados/tratados/agrupamento_empreend_acidentes_por_municipio_e_ano.csv', index=False)